# [추천시스템] LightGCN with Cornac

* LightGCN with Cornac: 성능 개선 실험 및 실습
* 공식 Cornac LightGCN을 사용한 종합 실험 코드

* 필요 패키지 설치

In [63]:
! pip install cornac

In [64]:
! pip install -U dgl -f https://data.dgl.ai/wheels/torch-2.3/cu121/repo.html

Looking in links: https://data.dgl.ai/wheels/torch-2.3/cu121/repo.html


* 환경 확인

In [65]:
!python --version

Python 3.12.11


In [66]:
import torch
print("PyTorch 버전 : ", torch.__version__)
print("Cuda 버전 : ", torch.version.cuda)
import cornac
print("cornac 버전 : ", cornac.__version__)
import dgl
print("dgl 버전 : ", dgl.__version__)


PyTorch 버전 :  2.8.0+cu126
Cuda 버전 :  12.6
cornac 버전 :  2.3.4
dgl 버전 :  2.5.0+cu121


### 1. 임포트 및 초기 설정

In [67]:
import numpy as np
import cornac
from cornac.data import Reader
from cornac.eval_methods import RatioSplit
from cornac.models import LightGCN
from cornac.metrics import NDCG, Recall, Precision, MAE, RMSE
import pandas as pd
from datetime import datetime

print("="*60)
print("LightGCN 추천 시스템 구현 및 성능 개선 실험")
print("Cornac Framework 사용")
print("="*60)

LightGCN 추천 시스템 구현 및 성능 개선 실험
Cornac Framework 사용


### 2. 데이터 로드 및 전처리

* 데이터 로드

In [68]:

print("\n[Step 1] 데이터 로드 중...")
# Cornac에서 제공하는 MovieLens 100K 데이터셋 사용
from cornac.datasets import movielens

ml_100k = movielens.load_feedback(variant="100K")
print(f"✓ 데이터 로드 완료: {len(ml_100k)} 개의 평점 데이터")

# 컬럼명을 지정하여 DataFrame으로 변환
df = pd.DataFrame(ml_100k, columns=["user_id", "item_id", "rating"])

# 데이터 확인
display(df.head())


[Step 1] 데이터 로드 중...
✓ 데이터 로드 완료: 100000 개의 평점 데이터


,user_id,item_id,rating
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0


* 데이터 분할

In [69]:

# 데이터 분할: Train 80%, Test 20%
ratio_split = RatioSplit(
    data=ml_100k,
    test_size=0.2, # 전체의 20%를 테스트 데이터로, 나머지 80%를 학습 데이터로 사용.
    rating_threshold=4.0,  # 4점 이상을 긍정적 상호작용으로 간주, 추천 모델이 “이 사용자가 이 아이템을 좋아한다”고 학습할 근거로 사용됨.
    seed=42, #  난수 시드를 고정해 매번 같은 결과가 나오도록 함.
    verbose=True # 실행 시 데이터 통계를 화면에 출력.
)

print(f"✓ 학습 데이터: {len(ratio_split.train_set.uir_tuple[0])} 개")
print(f"✓ 테스트 데이터: {len(ratio_split.test_set.uir_tuple[0])} 개")


rating_threshold = 4.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1651
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1651
Number of ratings = 19964
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 943
Total items = 1651
✓ 학습 데이터: 80000 개
✓ 테스트 데이터: 19964 개


### 3. 평가 지표 설정

In [70]:
print("\n[Step 2] 평가 지표 설정...")

metrics = [
    NDCG(k=10),      # Normalized Discounted Cumulative Gain
    NDCG(k=20),
    Recall(k=10),    # Recall at k
    Recall(k=20),
    Precision(k=10), # Precision at k
    Precision(k=20),
    RMSE(),          # Root Mean Square Error
    MAE()            # Mean Absolute Error
]

print("✓ 평가 지표: NDCG, Recall, Precision, RMSE, MAE")


[Step 2] 평가 지표 설정...
✓ 평가 지표: NDCG, Recall, Precision, RMSE, MAE


### 4. LightGCN 모델 구성 (기본 모델)

In [71]:
BATCH_SIZE = 1024

In [74]:

print("\n[Step 3] 기본 LightGCN 모델 설정...")

baseline_model = LightGCN(
    name="LightGCN_Baseline",
    emb_size=64, # 유저/아이템 임베딩 벡터의 차원 (예: 64차원)
    num_layers=3, # LightGCN의 그래프 전파(layer) 깊이. 클수록 더 넓은 연결 정보를 학습
    learning_rate=0.001, # 옵티마이저의 학습률
    lambda_reg=1e-5, # 과적합 방지를 위한 L2 정규화 계수
    num_epochs=20, # 전체 학습 데이터를 몇 번 반복할지
    batch_size=BATCH_SIZE, # 한 번에 학습에 사용하는 데이터 수
    verbose=True, # 학습 과정을 콘솔에 출력할지 여부
    seed=42 # 랜덤 초기화를 고정해 결과 재현성을 확보
)


[Step 3] 기본 LightGCN 모델 설정...


### 5. 성능 개선 실험: 하이퍼파라미터 튜닝

In [75]:
print("\n[Step 4] 성능 개선 실험 시작...")
print("다양한 하이퍼파라미터 조합으로 실험 진행\n")


# 실험할 모델 구성
experiment_models = [
    # Baseline
    LightGCN(
        name="Baseline",
        emb_size=64,
        num_layers=3,
        learning_rate=0.001,
        lambda_reg=1e-5,
        num_epochs=20,
        batch_size=BATCH_SIZE,
        seed=42
    ),

    # Experiment 1: 더 큰 Embedding Size
    LightGCN(
        name="Large_Embedding",
        emb_size=96,
        num_layers=3,
        learning_rate=0.001,
        lambda_reg=1e-5,
        num_epochs=20,
        batch_size=BATCH_SIZE,
        seed=42
    ),

    # Experiment 2: 더 많은 Layer
    LightGCN(
        name="Deep_Layers",
        emb_size=64,
        num_layers=5,
        learning_rate=0.001,
        lambda_reg=1e-5,
        num_epochs=20,
        batch_size=BATCH_SIZE,
        seed=42
    ),

    # Experiment 3: 높은 Learning Rate
    LightGCN(
        name="High_LR",
        emb_size=64,
        num_layers=3,
        learning_rate=0.003,
        lambda_reg=1e-5,
        num_epochs=20,
        batch_size=BATCH_SIZE,
        seed=42
    ),

    # Experiment 4: 강한 Regularization
    LightGCN(
        name="Strong_Reg",
        emb_size=64,
        num_layers=3,
        learning_rate=0.001,
        lambda_reg=1e-3,
        num_epochs=20,
        batch_size=BATCH_SIZE,
        seed=42
    ),

    # Experiment 5: 최적 조합 (큰 임베딩 + 적절한 Layer)
    LightGCN(
        name="Optimized",
        emb_size=128,
        num_layers=3,
        learning_rate=0.003,
        lambda_reg=1e-4,
        num_epochs=20,
        batch_size=BATCH_SIZE,
        seed=42
    ),
]


[Step 4] 성능 개선 실험 시작...
다양한 하이퍼파라미터 조합으로 실험 진행



### 6. 모델 학습 및 평가

In [76]:
print("\n[Step 5] 모든 모델 학습 및 평가...")
hyper_params_results = []
metrics_results = []

for i, model in enumerate(experiment_models, 1):
    print(f"\n{'='*60}")
    print(f"실험 {i}/{len(experiment_models)}: {model.name}")
    print(f"{'='*60}")

    # Cornac Experiment 실행
    experiment = cornac.Experiment(
        eval_method=ratio_split,
        models=[model],
        metrics=metrics,
        user_based=True
    )

    # 모델 학습 및 평가
    experiment.run()
    # ---------------------------
    # 결과를 DataFrame으로 안전하게 수집
    # ---------------------------
    rows = []
    for res in experiment.result:
        row = {"Model": res.model_name}
        # 주요 지표들 (버전 공통)
        row.update(res.metric_avg_results)
        rows.append(row)

    df_results = pd.DataFrame(rows).set_index("Model").sort_index(axis=1)
    metrics_results.append(df_results)

    hyper_params_result = {
        'Model': model.name,
        'Embedding_Size': model.emb_size,
        'N_Layers': model.num_layers,
        'Learning_Rate': model.learning_rate,
        'Lambda_Reg': model.lambda_reg
    }
    hyper_params_results.append(result)


[Step 5] 모든 모델 학습 및 평가...

실험 1/6: Baseline

[Baseline] Training started!

[Baseline] Evaluation started!


Rating:   0%|          | 0/19964 [00:00<?, ?it/s]

Ranking:   0%|          | 0/940 [00:00<?, ?it/s]


TEST:
...
         |    MAE |   RMSE | NDCG@10 | NDCG@20 | Precision@10 | Precision@20 | Recall@10 | Recall@20 | Train (s) | Test (s)
-------- + ------ + ------ + ------- + ------- + ------------ + ------------ + --------- + --------- + --------- + --------
Baseline | 1.2394 | 1.4844 |  0.1915 |  0.2101 |       0.1438 |       0.1191 |    0.1581 |    0.2508 |   41.0289 |   1.0916


실험 2/6: Large_Embedding

[Large_Embedding] Training started!

[Large_Embedding] Evaluation started!


Rating:   0%|          | 0/19964 [00:00<?, ?it/s]

...

Ranking:   0%|          | 0/940 [00:00<?, ?it/s]


TEST:
...
           |    MAE |   RMSE | NDCG@10 | NDCG@20 | Precision@10 | Precision@20 | Recall@10 | Recall@20 | Train (s) | Test (s)
---------- + ------ + ------ + ------- + ------- + ------------ + ------------ + --------- + --------- + --------- + --------
Strong_Reg | 1.2467 | 1.4904 |  0.1911 |  0.2106 |       0.1433 |       0.1194 |    0.1566 |    0.2514 |   40.4937 |   1.0427


실험 6/6: Optimized

[Optimized] Training started!

[Optimized] Evaluation started!


Rating:   0%|          | 0/19964 [00:00<?, ?it/s]

Ranking:   0%|          | 0/940 [00:00<?, ?it/s]


TEST:
...
          |    MAE |   RMSE | NDCG@10 | NDCG@20 | Precision@10 | Precision@20 | Recall@10 | Recall@20 | Train (s) | Test (s)
--------- + ------ + ------ + ------- + ------- + ------------ + ------------ + --------- + --------- + --------- + --------
Optimized | 1.2125 | 1.4821 |  0.2416 |  0.2642 |       0.1775 |       0.1446 |    0.2008 |    0.3084 |   40.8005 |   1.0893



### 7. 결과 분석 및 시각화

In [77]:
def combine_metrics(metrics_results, sort_by="NDCG@20", descending=True, ndigits=4):
    """여러 결과 DF를 하나로 병합하고 보기 좋게 정리"""
    # 1) 유효한 DF만 추려서 병합
    dfs = [df for df in metrics_results if isinstance(df, pd.DataFrame) and not df.empty]
    if not dfs:
        raise ValueError("metrics_results가 비어 있습니다. 실험 실행 후 수집된 DataFrame을 넣어주세요.")
    df = pd.concat(dfs, axis=0, sort=False)

    # 2) 같은 모델 이름이 여러 번 있으면 마지막 결과로
    df = df[~df.index.duplicated(keep="last")]

    # 3) 컬럼 순서 정리 (있는 컬럼만 적용)
    order = [
        "NDCG@10","NDCG@20","Recall@10","Recall@20",
        "Precision@10","Precision@20","RMSE","MAE","Train (s)","Test (s)","Total (s)"
    ]
    cols = [c for c in order if c in df.columns]
    df = df[cols].astype(float).round(ndigits)
    df.index.name = "Model"

    # 4) 정렬 (기본: NDCG@20 내림차순)
    if sort_by in df.columns:
        df = df.sort_values(by=sort_by, ascending=not descending)

    return df

def style_metrics(df):
    """가독성 좋은 하이라이트 스타일"""
    better_high = [c for c in ["NDCG@10","NDCG@20","Recall@10","Recall@20","Precision@10","Precision@20"] if c in df.columns]
    better_low  = [c for c in ["RMSE","MAE","Train (s)","Test (s)","Total (s)"] if c in df.columns]
    return (df.style
              .format("{:.4f}")
              .set_properties(**{"text-align":"center"})
              .highlight_max(subset=better_high, axis=0)   # 성능은 클수록 좋은 지표
              .highlight_min(subset=better_low,  axis=0)) # 오차/시간은 작을수록 좋음

# 사용 예시 -------------------------------------------------
df_metrics = combine_metrics(metrics_results, sort_by="NDCG@20", descending=True, ndigits=4)
display(style_metrics(df_metrics))        # 깔끔한 표로 출력

,NDCG@10,NDCG@20,Recall@10,Recall@20,Precision@10,Precision@20,RMSE,MAE,Train (s),Test (s)
Model,,,,,,,,,,
Optimized,0.2416,0.2642,0.2008,0.3084,0.1775,0.1446,1.4821,1.2125,40.8005,1.0893
High_LR,0.2325,0.2540,0.1963,0.2997,0.1706,0.1385,1.4707,1.2065,40.6479,1.0633
Strong_Reg,0.1911,0.2106,0.1566,0.2514,0.1433,0.1194,1.4904,1.2467,40.4937,1.0427
Baseline,0.1915,0.2101,0.1581,0.2508,0.1438,0.1191,1.4844,1.2394,41.0289,1.0916
Large_Embedding,0.1889,0.2096,0.1516,0.2470,0.1405,0.1196,1.4851,1.2349,40.3079,1.1424
Deep_Layers,0.1813,0.2011,0.1493,0.2357,0.1290,0.1094,1.5875,1.3568,45.3059,1.0694


### 8. 추천 예시

In [78]:
# ============================================================
# 7. 추천 예시 (점수 없이, 안전 출력)
# ============================================================
print("\n[Step 7] 추천 시스템 작동 예시...")

final_model = experiment_models[-1]

# 모델이 같은 split으로 학습되어 있는지 확인
if not hasattr(final_model, "train_set") or final_model.train_set is not ratio_split.train_set:
    print("모델 재학습 중...")
    final_model.fit(ratio_split.train_set)

model_train_set = final_model.train_set
print(f"\n학습 데이터의 사용자 수: {model_train_set.num_users}")
print(f"학습 데이터의 아이템 수: {model_train_set.num_items}")

available_users = list(model_train_set.uid_map.keys())[:5]
top_k = 10

def to_raw_item_id(item_idx_or_raw):
    # recommend가 내부 인덱스를 줄 수도, raw를 줄 수도 있어 둘 다 처리
    try:
        # 내부 인덱스(int)면 raw로 변환 시도
        return model_train_set.iid_map.reverse(item_idx_or_raw)
    except Exception:
        # 이미 raw면 그대로 반환
        return item_idx_or_raw

for i, user_raw in enumerate(available_users, 1):
    print(f"\n[예시 {i}] 사용자 '{user_raw}'에 대한 Top-{top_k} 추천:")
    try:
        recs = final_model.recommend(user_raw, k=top_k)   # raw 사용자 ID 그대로
        # recs는 보통 ["itemA","itemB",...] 또는 [123, 45, ...] 형태
        for rank, item in enumerate(recs, 1):
            item_raw = to_raw_item_id(item)
            print(f"  {rank}. 아이템 ID: {item_raw}")
    except Exception as e:
        print(f"  오류 발생: {e}")


[Step 7] 추천 시스템 작동 예시...

학습 데이터의 사용자 수: 943
학습 데이터의 아이템 수: 1651

[예시 1] 사용자 '877'에 대한 Top-10 추천:
  1. 아이템 ID: 286
  2. 아이템 ID: 50
  3. 아이템 ID: 302
  4. 아이템 ID: 258
  5. 아이템 ID: 100
  6. 아이템 ID: 269
  7. 아이템 ID: 313
  8. 아이템 ID: 127
  9. 아이템 ID: 174
  10. 아이템 ID: 56

[예시 2] 사용자 '815'에 대한 Top-10 추천:
  1. 아이템 ID: 174
  2. 아이템 ID: 98
  3. 아이템 ID: 69
  4. 아이템 ID: 172
  5. 아이템 ID: 195
  6. 아이템 ID: 423
  7. 아이템 ID: 210
  8. 아이템 ID: 56
  9. 아이템 ID: 204
  10. 아이템 ID: 168

[예시 3] 사용자 '94'에 대한 Top-10 추천:
  1. 아이템 ID: 56
  2. 아이템 ID: 174
  3. 아이템 ID: 172
  4. 아이템 ID: 195
  5. 아이템 ID: 98
  6. 아이템 ID: 69
  7. 아이템 ID: 210
  8. 아이템 ID: 50
  9. 아이템 ID: 403
  10. 아이템 ID: 204

[예시 4] 사용자 '416'에 대한 Top-10 추천:
  1. 아이템 ID: 258
  2. 아이템 ID: 50
  3. 아이템 ID: 294
  4. 아이템 ID: 181
  5. 아이템 ID: 300
  6. 아이템 ID: 313
  7. 아이템 ID: 288
  8. 아이템 ID: 328
  9. 아이템 ID: 121
  10. 아이템 ID: 117

[예시 5] 사용자 '500'에 대한 Top-10 추천:
  1. 아이템 ID: 100
  2. 아이템 ID: 237
  3. 아이템 ID: 1
  4. 아이템 ID: 50
  5. 아이템 ID: 275
  6. 아이템 ID: 1

### 9. LightGCN vs NGCF vs MF 비교 실험

In [80]:
import time, pandas as pd
from cornac.experiment import Experiment
from cornac.models import LightGCN, NGCF, MF

# 1) 모델 정의
lgn = LightGCN(
    name="LightGCN_Best",
    emb_size=128, num_layers=3, learning_rate=0.003,
    lambda_reg=1e-4, num_epochs=20, batch_size=BATCH_SIZE,
    verbose=True, seed=42
)

ngcf = NGCF(
    name="NGCF",
    emb_size=64,
    layer_sizes=[64, 64, 64],        # 레이어별 임베딩 크기
    dropout_rates=[0.1, 0.1, 0.1],   # 메시지 전파 dropout
    learning_rate=0.001,
    lambda_reg=1e-5,
    num_epochs=20,
    batch_size=BATCH_SIZE,
    early_stopping={"min_delta": 1e-4, "patience": 50},
    verbose=True, seed=42
)

mf = MF(
    name="MF_Explicit",
    k=64, max_iter=50, learning_rate=0.01,
    lambda_reg=0.001, use_bias=True,
    early_stop=True, verbose=True, seed=42
)

models = [lgn, ngcf, mf]

# 2) 같은 분할/같은 지표로 실험 실행
t0 = time.perf_counter()
exp = Experiment(eval_method=ratio_split, models=models, metrics=metrics, user_based=True)
exp.run()
total_runtime = time.perf_counter() - t0

# 3) 결과 → DataFrame (버전 호환 안전 수집)
rows = []
for r in exp.result:
    row = {"Model": r.model_name, **r.metric_avg_results}
    # 시간 필드는 버전에 따라 없을 수 있으므로 있으면만 기록
    for k in ("train_time","training_time","fit_time"):
        if hasattr(r, k): row["Train (s)"] = getattr(r, k)
    for k in ("test_time","evaluation_time","eval_time"):
        if hasattr(r, k): row["Test (s)"] = getattr(r, k)
    if "Train (s)" not in row and "Test (s)" not in row:
        row["Total (s)"] = total_runtime
    rows.append(row)

df_cmp = pd.DataFrame(rows).set_index("Model")

# 4) 보기 좋게 정리/정렬/표시
order = ["NDCG@10","NDCG@20","Recall@10","Recall@20",
         "Precision@10","Precision@20","RMSE","MAE","Train (s)","Test (s)","Total (s)"]
cols = [c for c in order if c in df_cmp.columns]
df_cmp = df_cmp[cols].round(4).sort_values(by="NDCG@20", ascending=False)

display(df_cmp)          # 노트북 표 출력



[LightGCN_Best] Training started!


Training:   0%|          | 0/20 [00:00<?, ?iter/s]

Epoch:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch:   0%|          | 0/79 [00:00<?, ?it/s]

...

Optimization finished!

[MF_Explicit] Evaluation started!


Rating:   0%|          | 0/19964 [00:00<?, ?it/s]

Ranking:   0%|          | 0/940 [00:00<?, ?it/s]


TEST:
...
              |    MAE |   RMSE | NDCG@10 | NDCG@20 | Precision@10 | Precision@20 | Recall@10 | Recall@20 | Train (s) | Test (s)
------------- + ------ + ------ + ------- + ------- + ------------ + ------------ + --------- + --------- + --------- + --------
LightGCN_Best | 1.2125 | 1.4821 |  0.2416 |  0.2642 |       0.1775 |       0.1446 |    0.2008 |    0.3084 |   43.6752 |   1.1013
NGCF          | 1.6810 | 1.8869 |  0.1774 |  0.2038 |       0.1427 |       0.1264 |    0.1390 |    0.2444 |   75.8338 |   1.1406
MF_Explicit   | 0.7916 | 0.9791 |  0.0872 |  0.0924 |       0.0724 |       0.0611 |    0.0584 |    0.0970 |    0.3536 |   1.1129



,NDCG@10,NDCG@20,Recall@10,Recall@20,Precision@10,Precision@20,RMSE,MAE,Train (s),Test (s)
Model,,,,,,,,,,
LightGCN_Best,0.2416,0.2642,0.2008,0.3084,0.1775,0.1446,1.4821,1.2125,43.6752,1.1013
NGCF,0.1774,0.2038,0.1390,0.2444,0.1427,0.1264,1.8869,1.6810,75.8338,1.1406
MF_Explicit,0.0872,0.0924,0.0584,0.0970,0.0724,0.0611,0.9791,0.7916,0.3536,1.1129
